In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
np.random.seed(123)

In [2]:
train_path = os.path.join('raw_data', 'application_train.csv')
train = pd.read_csv(train_path, index_col='SK_ID_CURR')
train.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train.info()

In [ ]:
categ_columns = list(train.select_dtypes(include='object').columns)
categ_columns

In [ ]:
from ipywidgets import interact, fixed
def plot_categ_interact(x, y, data):
    print(x)
    percent_nans = np.round(100*np.sum(data[x].isnull())/data.shape[0], 2)
    if percent_nans >0:
        print('the col {} has {} percent nans in the data'.format(x, percent_nans))
#     return sns.stripplot(x=x, y=y, data=data, jitter=True)
    p = sns.countplot(x, hue=y, data=data);
    p.set_xticklabels(p.get_xticklabels(), rotation=45)
    return p


interact(plot_categ_interact, x=categ_columns, y=fixed('TARGET'), data=fixed(train));

In the column CODE_GENDER the value XNA looks weird. (4 instances, we can drop the or make them nans).

'NAME_EDUCATION_TYPE' and 'WEEKDAY_APPR_PROCESS_START' can be ordinal.

In the column 'NAME_FAMILY_STATUS' we can make more features, like 'Was/Is married'.

'ORGANIZATION_TYPE' needs a further look.

'FONDKAPREMONT_MODE' this is mainly nan, maybe it's not so good. for now, all nan are same as "not specified".

'EMERGENCYSTATE_MODE' maybe all the nan's are 'No'. For now I'll leave it as nan

In [ ]:
# train.ORGANIZATION_TYPE.value_counts()
# list(train.NAME_FAMILY_STATUS.value_counts(dropna=False).index)

In [3]:
def get_ORGANIZATION_TYPE_higher_level(value):
    for t in ['Business Entity', 'Trade', 'Transport', 'Industry']:
        if t in value:
            return t

def col_convert(in_data):
    data = in_data.copy()
    data['Was_Married'] = data.NAME_FAMILY_STATUS.apply(lambda v: v in ['Separated', 'Widow'])
    data['IS_Married'] = data.NAME_FAMILY_STATUS.apply(lambda v: v in ['Married', 'Civil marriage'])
    
    
    data['FONDKAPREMONT_MODE'] = data['FONDKAPREMONT_MODE'].fillna('not specified')
    data['FLAG_OWN_REALTY'] = data['FLAG_OWN_REALTY'].apply(lambda v: v=='Y')
    data['FLAG_OWN_CAR'] = data['FLAG_OWN_CAR'].apply(lambda v: v=='Y')

    data['CODE_GENDER'] = data['CODE_GENDER'].replace({'M':True, 'F':False, 'XNA':None})
    data['NAME_CONTRACT_TYPE'] = data['NAME_CONTRACT_TYPE'].apply(lambda v: v=='Cash loans')
    data['EMERGENCYSTATE_MODE'] = data['EMERGENCYSTATE_MODE'].apply(lambda v: v=='Yes')
    
    data['NAME_EDUCATION_TYPE'] = data['NAME_EDUCATION_TYPE'].replace({'Secondary / secondary special':2,
                                                                       'Higher education':4, 'Incomplete higher':3,
                                                                       'Lower secondary':1, 'Academic degree':5})
    
    data['IS_WEEKEND_APPR_PROCESS_START'] = data['WEEKDAY_APPR_PROCESS_START'].replace({'TUESDAY':0, 'WEDNESDAY':0, 
                                                                                     'MONDAY':0, 'THURSDAY':0,
                                                                                     'FRIDAY':1, 'SATURDAY':1,
                                                                                     'SUNDAY':1}) 
    
    data['WEEKDAY_APPR_PROCESS_START'] = data['WEEKDAY_APPR_PROCESS_START'].replace({'TUESDAY':3, 'WEDNESDAY':4, 
                                                                                     'MONDAY':2, 'THURSDAY':3,
                                                                                     'FRIDAY':6, 'SATURDAY':7,
                                                                                     'SUNDAY':1}) #Im not sure this is right
    
    data['ORGANIZATION_TYPE_upper_level'] = data['ORGANIZATION_TYPE'].apply(get_ORGANIZATION_TYPE_higher_level)
    
    data['EXT_SOURCE_sum'] = data[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].sum(axis=1)
    data['EXT_SOURCE_mean'] = data[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    
    data['DOCUMENT_count'] = data[['FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6',
                                   'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11',
                                   'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16',
                                   'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21',]].sum(axis=1)

    return data

In [4]:
neg_pos = np.sum(train.TARGET == 0) / np.sum(train.TARGET == 1)
neg_pos

11.387150050352467

In [ ]:
sns.countplot(train.TARGET); # The data is very unballanced
# np.sum(train.TARGET == 1) / train.TARGET.shape[0]


# "Bureau" Table

In [5]:
bureau_path = os.path.join('raw_data', 'bureau.csv')
bureau = pd.read_csv(bureau_path, index_col='SK_ID_CURR')
bureau.head()

/home/ariel/venv/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
SK_ID_CURR,,,,,,,,,,,,,,,,
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [17]:
def count_existance(row, new_table):
    if set(row.index) in list(new_table.index):
        return new_table.loc[row.index, :].shape[0]
    return 0

def add_table_summerized(data, new_table, table_name, matching_index_col=None):
    df = data.copy()

    df['count_{}_instances'.format(table_name)] = df.apply(lambda row: count_existance(row, new_table))
    for num_col in new_table.select_dtypes(exclude=['object', 'category']).columns:
        if num_col == matching_index_col: continue
        df = df.join(new_table.groupby(new_table.index.name)[num_col].sum().rename('{}_sum'.format(num_col)),
                     rsuffix=table_name, on=matching_index_col)
        df = df.join(new_table.groupby(new_table.index.name)[num_col].mean().rename('{}_mean'.format(num_col)),
                     rsuffix=table_name, on=matching_index_col)
        df = df.join(new_table.groupby(new_table.index.name)[num_col].median().rename('{}_median'.format(num_col)),
                     rsuffix=table_name, on=matching_index_col)
        df = df.join(new_table.groupby(new_table.index.name)[num_col].std().rename('{}_std'.format(num_col)),
                     rsuffix=table_name, on=matching_index_col)
    
    for categ_col in new_table.select_dtypes(include='object').columns:
        if categ_col == matching_index_col: continue
        dummie_cols = pd.get_dummies(new_table[categ_col], prefix=categ_col).groupby(new_table.index.name).sum()
        df = df.join(dummie_cols, rsuffix=table_name, on=matching_index_col)
        
#         dummies_percent = dummie_cols/df['count_{}_instances'.format(table_name)]
#         for col in dummies_percent.columns:
#             dummies_precent[col].rename('{}_percent'.format(col), inplace=True)
#         df = df.join(dummies_percent, rsuffix=table_name)
    
    if matching_index_col == None:
        return df
    return df.drop(matching_index_col, axis=1)

# "Bureau balance" Table

In [7]:
bureau_balance_path = os.path.join('raw_data', 'bureau_balance.csv')
bureau_balance = pd.read_csv(bureau_balance_path, index_col='SK_ID_BUREAU')
bureau_balance.head()

/home/ariel/venv/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,MONTHS_BALANCE,STATUS
SK_ID_BUREAU,,
5715448,0,C
5715448,-1,C
5715448,-2,C
5715448,-3,C
5715448,-4,C


# "previous_application" Table

In [ ]:
previous_application_path = os.path.join('raw_data', 'previous_application.csv')
previous_application = pd.read_csv(previous_application_path, index_col='SK_ID_CURR')
previous_application.head()

In [ ]:
previous_application.NAME_YIELD_GROUP.value_counts()
previous_application['NAME_YIELD_GROUP'] = previous_application['NAME_YIELD_GROUP'].replace({'XNA':0, })

# Predict

In [10]:
from sklearn import metrics
# from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.preprocessing import Imputer, StandardScaler, RobustScaler
import xgboost as xgb
# from sklearn.svm import SVR, LinearSVR
# from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
# from scipy.special import boxcox1p, inv_boxcox1p
import datetime
# from imblearn import combine, over_sampling
# from imblearn import pipeline as imblearn_pipeline
from imblearn.ensemble import BalancedBaggingClassifier

import category_encoders
np.random.seed(123)


In [11]:
import multiprocessing
def model_run(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train) 
    test_pred = model.predict(X_test)

    train_auc = metrics.roc_auc_score(y_train, train_pred)
    test_auc = metrics.roc_auc_score(y_test, test_pred)

    return train_auc, test_auc

def test_on_train(model, X, y):
    input_to_multi = []
    start = datetime.datetime.now()
    for train_index, test_index in RepeatedStratifiedKFold(n_splits=5, n_repeats=4, random_state=222).split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        input_to_multi.append([model, X_train, X_test, y_train, y_test])

    with multiprocessing.Pool() as p:
        KFold_results = p.starmap(model_run, input_to_multi)


    print(datetime.datetime.now() - start)
    return pd.DataFrame(KFold_results,
                        columns=['train_auc', 'test_auc']).mean(axis=0)


In [12]:
def test_model(X, y, model):
    start = datetime.datetime.now()
    train_auc = []
    test_auc = []
    for train_index, test_index in RepeatedStratifiedKFold(n_splits=5, n_repeats=4, random_state=123).split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        
        model.fit(X_train, y_train)
        train_pred = model.predict(X_train)
        test_pred = model.predict(X_test)
        
        train = metrics.roc_auc_score(y_train, train_pred)
        test = metrics.roc_auc_score(y_test, test_pred)
        
        train_auc.append(train)
        test_auc.append(test)
        

    print(datetime.datetime.now() - start) 
    return train_auc, test_auc

In [8]:
bureau_new = add_table_summerized(bureau, bureau_balance, 'Bureau balance', matching_index_col='SK_ID_BUREAU')
bureau_new.head()

,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,MONTHS_BALANCE_median,MONTHS_BALANCE_std,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
215354,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215354,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215354,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215354,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215354,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
np.random.seed(123)
start = datetime.datetime.now()
print(start)
X = col_convert(train).drop('TARGET', axis=1)
y = train['TARGET']

X_new = add_table_summerized(X, bureau_new, 'bureau')
print(datetime.datetime.now() - start)

2018-07-20 18:45:54.002851
0:00:38.729446


In [19]:
categorical_cols = list(X_new.select_dtypes(include='object').columns)
print(categorical_cols)
X_train, X_test, y_train, y_test = train_test_split(X_new, y, stratify=y)

['CODE_GENDER', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'ORGANIZATION_TYPE_upper_level']


In [20]:
len(list(X_new.columns))

251

In [21]:
np.random.seed(123)
start = datetime.datetime.now()
model = Pipeline([
# model = imblearn_pipeline.Pipeline([
    ('categ', category_encoders.binary.BinaryEncoder(cols=categorical_cols,
                                                      drop_invariant=True, impute_missing=True)),
    ('impute', Imputer()),
    ('scale', StandardScaler()),
# #     ('resample', combine.SMOTEENN(random_state=123)),
# #     ('resample', combine.SMOTETomek()),
    ('xgb', xgb.XGBClassifier(eval_metric='auc', learning_rate=0.1, max_depth=6, booster='dart',reg_lambda=0.1,
#                               sample_type='weighted',
                              scale_pos_weight=neg_pos))
#     ('resample', over_sampling.SMOTE()),
#     ('logreg', LogisticRegression())
#     ('resample_Ensemble', BalancedBaggingClassifier(base_estimator=LogisticRegression(), n_estimators=10,
#                                                     max_samples=1.0, max_features=1.0, bootstrap=True,
#                                                     bootstrap_features=False, oob_score=False, warm_start=False,
#                                                     ratio='auto', replacement=False, n_jobs=3, random_state=123))
])

# print(test_on_train(model, X, y))
model.fit(X_train, y_train)
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

print(metrics.roc_auc_score(y_train, train_pred))
print(metrics.roc_auc_score(y_test, test_pred))
print(metrics.classification_report(y_test, test_pred))
print(datetime.datetime.now() - start)

/home/ariel/venv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.743820878142073
0.6900520301363985
             precision    recall  f1-score   support

          0       0.96      0.73      0.83     70672
          1       0.17      0.65      0.28      6206

avg / total       0.90      0.72      0.78     76878

0:06:39.752483


/home/ariel/venv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
from sklearn.model_selection import GridSearchCV
params = {
#     'xgb__max_depth':np.arange(4, 9),
#     'xgb__normalize_type':['tree', 'forest'],
    'xgb__sample_type':['uniform', 'weighted'],
    'xgb__reg_lambda':[0.4, 0.8, 1, 1.5, 2],
#          'xgb__colsample_bytree':np.arange(0.7, 1.0, 0.1),
#          'xgb__colsample_bylevel':np.arange(0.7, 1.0, 0.1),
    'xgb__objective':['reg:logistic', 'binary:logistic', 'binary:logitraw'],
#          'xgb__booster':['dart', 'gbtree', 'gblinear']
}
GCV = GridSearchCV(model, params, cv=4, verbose=2, n_jobs=3)
GCV.fit(X, y)

# predict on real Test

In [22]:
test = pd.read_csv(os.path.join('raw_data', 'application_test.csv'), index_col='SK_ID_CURR')
test.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
test_new = add_table_summerized(test, bureau_new, 'bureau')

In [24]:
model.fit(X_new, y)
all_train_pred = model.predict(X_new) 
print(metrics.roc_auc_score(y, all_train_pred))

0.7329673208451177


/home/ariel/venv/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [25]:
test_pred = model.predict_proba(col_convert(test_new))

In [26]:
submission = pd.DataFrame(list(zip(list(test.index) ,test_pred[:, 1])),
                          columns = ['SK_ID_CURR', 'TARGET']).set_index('SK_ID_CURR')
submission.head()

,TARGET
SK_ID_CURR,
100001,0.380805
100005,0.420172
100013,0.288472
100028,0.180979
100038,0.576458


In [27]:
submission.to_csv(os.path.join('Submission_files', 'submission_{}.csv'.format(str(datetime.datetime.now()).replace(' ', '-'))))